## Importing Data from Wikipedia

In [143]:
import pandas as pd
from string import ascii_uppercase as alphabet
import pickle

In [145]:
all_tables = pd.read_html('https://en.wikipedia.org/wiki/2022_FIFA_World_Cup')

In [164]:
# print(all_tables[18])
# for i, table in enumerate(all_tables):
#     print(f"Table {i}")
#     print(table.head())

# print(all_tables[16])
# all_tables[58]['Pld']

dict_table={}

for letter,i in zip(alphabet,range(18,68,7)):
    df = all_tables[i]
    # df.rename(columns={df.columns[1]:'Team'}, inplace=True)
    # df.pop('Qualification')
    dict_table[f'Group {letter}'] = df

for i in range(18,68,7):
    all_tables[i]['Pld']=0
    all_tables[i]['W']=0
    all_tables[i]['D']=0
    all_tables[i]['L']=0
    all_tables[i]['GF']=0
    all_tables[i]['Pts']=0
    all_tables[i]['GA']=0
    all_tables[i]['GD']=0
    all_tables[i].sort_values(by='Team', inplace=True)
    all_tables[i].reset_index(drop=True, inplace=True)
    all_tables[i]['Pos'] = all_tables[i].index + 1
# all_tables[9]

In [165]:
dict_table['Group A']

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,1,Ecuador,0,0,0,0,0,0,0,0
1,2,Netherlands,0,0,0,0,0,0,0,0
2,3,Qatar (H),0,0,0,0,0,0,0,0
3,4,Senegal,0,0,0,0,0,0,0,0


In [166]:
with open('dict_table', 'wb') as output:
    pickle.dump(dict_table,output)

In [167]:
from bs4 import BeautifulSoup
import requests

In [168]:
# Create a function to get all the matches from a particular World Cup

def get_matches(year,link):
    response = requests.get(link)
    content = response.text
    soup = BeautifulSoup(content, 'lxml')
    matches = soup.find_all('div',class_='footballbox')
    home = []
    away = []
    score = []
    
    for match in matches:
        home.append(match.find('th',class_='fhome').get_text())
        score.append(match.find('th',class_='fscore').get_text())
        away.append(match.find('th',class_='faway').get_text())
    
    football = {'home':home,'away':away,'score':score}
    df_football = pd.DataFrame(football)
    df_football['year'] = year

    return df_football
    
    

In [169]:
# Getting all the matches data from all the world cups
years = [1930,1934,1938,1950,1954,1958,1962,1966,1970,1974,1978,1982,1986,1990,1994,1998,2002,2006,2010,2014,2018]
fifa = [get_matches(year,link = f'https://en.wikipedia.org/wiki/{year}_FIFA_World_Cup') for year in years]

In [170]:
# Getting Historical Data for all the past World Cups

df_fifa = pd.concat(fifa, ignore_index=True)
#had one match with result as walkover, so removed it...
df_fifa.drop(37, inplace=True)
df_fifa.to_csv('FIFA_World_Cups_Historical_Data.csv',index=False)

In [171]:
# Getting the fixtures of 2022 World Cup

link_to_fixtures = 'https://web.archive.org/web/20221115040351/https://en.wikipedia.org/wiki/2022_FIFA_World_Cup'
df_fixtures = get_matches('2022',link = link_to_fixtures)
df_fixtures.to_csv('FIFA_2022_Fixtures.csv',index=False)

In [172]:
# DATA CLEANING

df_historical_data = pd.read_csv('FIFA_World_Cups_Historical_Data.csv')
df_fixtures_2022 = pd.read_csv('FIFA_2022_Fixtures.csv')


# there are blanks and spaces ahead and after the words in names of countries which need to be removed
df_fixtures_2022['home'] = df_fixtures_2022['home'].str.strip()
df_fixtures_2022['away'] = df_fixtures_2022['away'].str.strip()
df_fixtures_2022['away'] = df_fixtures_2022['away'].str.replace('\u2013', '-')
df_historical_data['home'] = df_historical_data['home'].str.strip()
df_historical_data['away'] = df_historical_data['away'].str.strip()


# finding the rows with score other than digits and hyphen
    # Replace en-dash (8211) with a regular hyphen (-)
df_historical_data['score'] = df_historical_data['score'].str.replace('\u2013', '-')
df_historical_data['score'] = df_historical_data['score'].str.strip()
df_historical_data['score'] = df_historical_data['score'].str.replace(" ", "")
df_historical_data[df_historical_data['score'].str.contains('[^\d-]')]


# Replacing the "After Extra Time" abbreviation from the scores
df_historical_data['score'] = df_historical_data['score'].str.replace('[^\d-]','',regex=True)


# Replacing the traditional Home-Away method with 2 separate columns to store home and away goals 
df_historical_data[['home_goals','away_goals']] = df_historical_data['score'].str.split('-',expand=True)
df_historical_data = df_historical_data.drop('score',axis=1)


# Renaming Columns and Changing Data Types
df_historical_data.rename(columns = {'home':'HomeTeam','away':'AwayTeam','year':'Year','home_goals':'HomeGoals','away_goals':'AwayGoals'},inplace=True)
df_historical_data = df_historical_data.astype({'HomeGoals':int,'AwayGoals':int,'Year':int})
df_historical_data['TotalGoals'] = df_historical_data['HomeGoals'] + df_historical_data['AwayGoals']

In [173]:
df_historical_data.to_csv('FIFA_World_Cups_Historical_Data_Cleaned.csv')
df_fixtures_2022.to_csv('FIFA_2022_Fixtures_Cleaned.csv')

## Building a Model

In [174]:
import pandas as pd
import pickle
from scipy.stats import poisson

In [175]:
dict_table = pickle.load(open('dict_table','rb'))
df_historical_data = pd.read_csv('FIFA_World_Cups_Historical_Data_Cleaned.csv')
df_fixtures_2022 = pd.read_csv('FIFA_2022_Fixtures_Cleaned.csv')

In [176]:
# Splitting the dataframe to Home and Away Dataframes
df_home = df_historical_data[['HomeTeam','HomeGoals','AwayGoals']]
df_away = df_historical_data[['AwayTeam','HomeGoals','AwayGoals']]

In [177]:
# Renaming the columns to get a dataframe with team name, goals for and goals against
df_home = df_home.rename(columns = {'HomeTeam':'Team','HomeGoals':'GoalsFor' ,'AwayGoals':'GoalsAgainst'})
df_away = df_away.rename(columns = {'AwayTeam':'Team','HomeGoals':'GoalsAgainst' ,'AwayGoals':'GoalsFor'})

In [178]:
# Concatenating Home and Away dataframes, grouping by team and calculating mean to get TEAM STRENGTH
df_team_strength = pd.concat([df_home,df_away],ignore_index=True).groupby('Team').mean()

### Creating a function to predict points of a team  
***Why Poisson Distribution?***  
_Conditions for Poisson distribution:_  
_1) The number of events can be counted_  
_2) The occurence of events are independent_  
_3) The rate at which the events occur is constant_  
_4) 2 events can't occur at exactly the same instant of time_  

***So, if a Goal is considered an event, it fits our situation perfectly.***

In [179]:
# predict_points function: takes the names of home and away teams as input and gives their points as output
def predict_points(home,away):
    if (home in df_team_strength.index) and (away in df_team_strength.index):
        #lambda = (goals scored by home team)*(goals conceded by away team)
        lambda_home = (df_team_strength.at[home,'GoalsFor'])*(df_team_strength.at[away,'GoalsAgainst'])
        lambda_away = (df_team_strength.at[away,'GoalsFor'])*(df_team_strength.at[home,'GoalsAgainst'])
        prob_home,prob_away,prob_draw = 0,0,0
        for i in range(0,11): #number of goals scored by home team
            for j in range(0,11): #number of goals scored by away team
                p = (poisson.pmf(i,lambda_home))*(poisson.pmf(j,lambda_away))
                if i==j:
                    prob_draw+=p
                elif i<j:
                    prob_away+=p
                else:
                    prob_home+=p
        points_home = 3*prob_home + prob_draw
        points_away = 3*prob_away + prob_draw
        return (points_home,points_away)
    else:
        return (0,0)

In [180]:
# testing the function
predict_points('France','Germany')

(1.1210688997944207, 1.6827940577620726)

## Predicting World Cup

### Group Stage

In [181]:
# Splitting the fixtures into group stage and subsequent knockout stages
df_fixture_group = df_fixtures_2022[:48].copy()
df_fixture_ro16 = df_fixtures_2022[48:56].copy()
df_fixture_quarters = df_fixtures_2022[56:60].copy()
df_fixture_semis = df_fixtures_2022[60:62].copy()
df_fixture_third_place = df_fixtures_2022[62:63].copy()
df_fixture_final = df_fixtures_2022[63:].copy()

In [182]:
# Run all the fixtures of group stage and updating the group table
for group in dict_table:
    teams = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group[df_fixture_group['home'].isin(teams)]
    for index,row in df_fixture_group_6.iterrows():
        home,away = row['home'],row['away']
        points_home, points_away = predict_points(home,away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away

    dict_table[group] = dict_table[group].sort_values('Pts',ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team','Pts']]
    dict_table[group] = dict_table[group].round(0)

C:\Users\Naimish\AppData\Local\Temp\ipykernel_1880\3925187797.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.87343108]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Naimish\AppData\Local\Temp\ipykernel_1880\3925187797.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.39716107]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
C:\Users\Naimish\AppData\Local\Temp\ipykernel_1880\3925187797.py:8: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2.78567341]' has dtype incompatible with int

In [127]:
# Show Updated Table
# dict_table

### Knock-outs: Round of 16

In [183]:
# Updating the ro16 table with the winners and runner-ups of each group
for group in dict_table:
    group_winner = dict_table[group].loc[0,'Team']
    group_runners_up = dict_table[group].loc[1,'Team']
    df_fixture_ro16.replace({f'Winners {group}': group_winner, f'Runners-up {group}': group_runners_up},inplace=True)
df_fixture_ro16['winner']='?'

In [184]:
df_fixture_ro16

,Unnamed: 0,home,away,score,year,winner
48,48,Netherlands,Wales,Match 49,2022,?
49,49,Argentina,Denmark,Match 50,2022,?
50,50,France,Poland,Match 52,2022,?
51,51,England,Senegal,Match 51,2022,?
52,52,Germany,Belgium,Match 53,2022,?
53,53,Brazil,Uruguay,Match 54,2022,?
54,54,Croatia,Spain,Match 55,2022,?
55,55,Portugal,Switzerland,Match 56,2022,?


In [185]:
# Creating a get_winner function
def get_winner(df_fixture_updated):
    for index,row in df_fixture_updated.iterrows():
        home,away = row['home'],row['away']
        points_home,points_away = predict_points(home,away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner'] = winner
    return df_fixture_updated

In [186]:
get_winner(df_fixture_ro16)

,Unnamed: 0,home,away,score,year,winner
48,48,Netherlands,Wales,Match 49,2022,Netherlands
49,49,Argentina,Denmark,Match 50,2022,Argentina
50,50,France,Poland,Match 52,2022,France
51,51,England,Senegal,Match 51,2022,England
52,52,Germany,Belgium,Match 53,2022,Germany
53,53,Brazil,Uruguay,Match 54,2022,Brazil
54,54,Croatia,Spain,Match 55,2022,Croatia
55,55,Portugal,Switzerland,Match 56,2022,Portugal


## Knock-outs: Quarter Finals

In [187]:
# Creating an update_table function
def update_table(prev_fixture,next_fixture):
    for index, row in prev_fixture.iterrows():
        winner = prev_fixture.loc[index,'winner']
        match = prev_fixture.loc[index,'score']
        next_fixture.replace({f'Winners {match}':winner},inplace=True)
    next_fixture['winner']='?'
    return next_fixture

In [188]:
update_table(df_fixture_ro16,df_fixture_quarters)

,Unnamed: 0,home,away,score,year,winner
56,56,Germany,Brazil,Match 58,2022,?
57,57,Netherlands,Argentina,Match 57,2022,?
58,58,Croatia,Portugal,Match 60,2022,?
59,59,England,France,Match 59,2022,?


In [189]:
get_winner(df_fixture_quarters)

,Unnamed: 0,home,away,score,year,winner
56,56,Germany,Brazil,Match 58,2022,Brazil
57,57,Netherlands,Argentina,Match 57,2022,Netherlands
58,58,Croatia,Portugal,Match 60,2022,Portugal
59,59,England,France,Match 59,2022,France


## Knock-outs: Semi Finals

In [190]:
update_table(df_fixture_quarters,df_fixture_semis)

,Unnamed: 0,home,away,score,year,winner
60,60,Netherlands,Brazil,Match 61,2022,?
61,61,France,Portugal,Match 62,2022,?


In [191]:
get_winner(df_fixture_semis)

,Unnamed: 0,home,away,score,year,winner
60,60,Netherlands,Brazil,Match 61,2022,Brazil
61,61,France,Portugal,Match 62,2022,France


## Knock-outs: Final

In [192]:
update_table(df_fixture_semis,df_fixture_final)

,Unnamed: 0,home,away,score,year,winner
63,63,Brazil,France,Match 64,2022,?


In [193]:
get_winner(df_fixture_final)

,Unnamed: 0,home,away,score,year,winner
63,63,Brazil,France,Match 64,2022,Brazil
